# Taipy 설치

Taipy를 설치하려면 그냥 `pip install` 하세요.

In [1]:
pip install taipy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pmdarima

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install requests

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


# 패키지 가져오기 및 초기화

In [4]:
from taipy.gui import Gui, Markdown, notify
from taipy import Config
import taipy as tp

import datetime as dt

from pmdarima import auto_arima

import pandas as pd
import numpy as np

import requests

In [5]:
from pandas.core.common import SettingWithCopyWarning
import warnings

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

data_country_date = None
selected_country = None

selected_scenario = None
scenario_selector = None

scenario_name = None

result = None

selected_points = []

data_province_displayed = None

api_year = None
api_pop = None
api_country = None

# Taipy Gui 기본
## 마크다운 구문

Taipy는 Markdown 구문을 사용하여 요소를 표시합니다. `#` 제목을 만들고 `*` 텍스트를 기울임꼴로 표시하고 `**` 굵게 표시합니다.

![](img/gui_basic_eng.png)

In [6]:
page_md = """
# Taipy

Test **here** to put some *markdown*

Click to access the [doc](https://docs.taipy.io/en/latest/)
"""

In [7]:
page = Markdown(page_md)

gui = Gui(page=page)
gui.run(dark_mode=False)

 * Server starting on http://127.0.0.1:5000
 * Serving Flask app 'Taipy'
 * Debug mode: off


## 시각적 요소
다양한 시각적 요소를 만듭니다. 구문은 각 시각적 요소에 대해 항상 동일합니다.  
`<|{value}|name_of_visual_element|property_1=value_of_property_1|...|>`
- [slider](https://docs.taipy.io/en/latest/manuals/gui/viselements/slider/) 만들기 `<|{value}|slider|>`

- [date](https://docs.taipy.io/en/latest/manuals/gui/viselements/date/) 만들기 `<|{value}|date|>`

- [selector](https://docs.taipy.io/en/latest/manuals/gui/viselements/selector/) 만들기 `<|{value}|selector|lov={list_of_values}|>` 


![](img/control.png)

In [8]:
slider_value = 0
date_value = None
selected_value = None

In [9]:
selector = ['Test 1', 'Test 2', 'Test 3']

control_md = """
## Controls

<|{slider_value}|slider|> <|{slider_value}|>

<|{date_value}|date|> <|{date_value}|>

<|{selected_value}|selector|lov={selector}|>  <|{selected_value}|>
"""

In [10]:
gui.stop()
page.set_content(control_md)
gui.run(port=5001)

Gui server has been stopped
 * Server starting on http://127.0.0.1:5001
 * Serving Flask app 'Taipy'
 * Debug mode: off


## 데이터 비주얼리제이션

다른 지역의 사망자, 확인된 사례 및 회복된 수에 대한 정보를 수집하는 데이터 세트는 대화형 대시보드를 만드는 데 사용됩니다.

In [11]:
path_to_data = "data/covid-19-all.csv"
data = pd.read_csv(path_to_data, low_memory=False)
data[-5:]

,Country/Region,Province/State,Latitude,Longitude,Confirmed,Recovered,Deaths,Date
1241947,Vietnam,NaN,14.058324,108.277199,1465.0,1325.0,35.0,2020-12-31
1241948,West Bank and Gaza,NaN,31.952200,35.233200,138004.0,117183.0,1400.0,2020-12-31
1241949,Yemen,NaN,15.552727,48.516388,2099.0,1394.0,610.0,2020-12-31
1241950,Zambia,NaN,-13.133897,27.849332,20725.0,18660.0,388.0,2020-12-31
1241951,Zimbabwe,NaN,-19.015438,29.154857,13867.0,11250.0,363.0,2020-12-31


In [12]:
def initialize_case_evolution(data, selected_country='France'):
    # Aggregation of the dataframe per Country/Region
    data_country_date = data.groupby(["Country/Region",'Date'])\
                            .sum()\
                            .reset_index()
    
    # a country is selected, here France by default
    data_country_date = data_country_date.loc[data_country_date['Country/Region']==selected_country]
    return data_country_date

In [13]:
data_country_date = initialize_case_evolution(data)
data_country_date.head()

,Country/Region,Date,Latitude,Longitude,Confirmed,Recovered,Deaths
18101,France,2020-01-24,14.6415,-56.3159,2.0,0.0,0.0
18102,France,2020-01-25,14.6415,-56.3159,3.0,0.0,0.0
18103,France,2020-01-26,14.6415,-56.3159,3.0,0.0,0.0
18104,France,2020-01-27,14.6415,-56.3159,3.0,0.0,0.0
18105,France,2020-01-28,14.6415,-56.3159,4.0,0.0,0.0


프랑스의 사망자수( Y 는 사망자수 , x 는 날짜 )의 변화를 보여주는 [차트](https://docs.taipy.io/en/latest/manuals/gui/viselements/chart/) 를 만듭니다 . 시각적 요소(차트)는 특정 속성(예: x , y , 유형 )이 있는 다른 요소와 동일한 구문을 사용합니다 . 다음은 [차트의 몇 가지 예](https://docs.taipy.io/en/release-1.1/manuals/gui/viselements/charts/bar/)입니다 . x 및 y 속성 은 표시할 데이터 프레임 열의 이름만 필요합니다.

![](img/simple_graph.png)

In [14]:
country_md = "<|{data_country_date}|chart|x=Date|y=Deaths|type=bar|>"

In [15]:
gui.stop()
page.set_content(country_md)
gui.run(port=5002)

Gui server has been stopped
 * Server starting on http://127.0.0.1:5002
 * Serving Flask app 'Taipy'
 * Debug mode: off


## 새 추적(traces) 추가 
- 그래프에 확인 및 회복된 사례 수(_확증_ 및 _회복_)를 사망자 수와 함께 추가합니다.
- y (및 x)는 더 많은 추적(`y[1]=`, `y[2]=`, `y[3]=`)을 추가하기 위해 이러한 방식으로 인덱싱될 수 있습니다.

![](img/multi_traces.png)

In [16]:
country_md = "<|{data_country_date}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|>"

In [17]:
gui.stop()
page.set_content(country_md)
gui.run(port=5003)

Gui server has been stopped
 * Server starting on http://127.0.0.1:5003
 * Serving Flask app 'Taipy'
 * Debug mode: off


## 개인화된 속성으로 그래프 스타일 지정 
_layout_ 사전(dictionnary)은 막대가 표시되는 방법을 지정합니다. 그것들은 '쌓일(stacked)'될 것입니다.

_options_ 사전(dictionnary)은 선택되지 않은 마커(maker)의 불투명도(opacity)를 변경합니다 .

이것들은 Plotly 속성입니다.

![](img/stack_chart.png)

In [18]:
layout = {'barmode':'stack'}
options = {"unselected":{"marker":{"opacity":0.5}}}
country_md = "<|{data_country_date}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|layout={layout}|options={options}|>"

In [19]:
gui.stop()
page.set_content(country_md)
gui.run(port=5004)

Gui server has been stopped
 * Server starting on http://127.0.0.1:5004
 * Serving Flask app 'Taipy'
 * Debug mode: off


## 데이터를 요약(sum up)하는 텍스트 추가 
[텍스트](https://docs.taipy.io/en/latest/manuals/gui/viselements/text/) 시각적 요소를 사용합니다 .

- 총 사망(Deaths)수를 추가합니다( _data_country_date_ 의 마지막 줄 ) .
- 총 회복(Recovered)수를 추가합니다( _data_country_date_ 의 마지막 줄 ) .
- 총 확진(Confirmed)수를 추가합니다( _data_country_date_ 의 마지막 줄 ) .

In [20]:
data_country_date

,Country/Region,Date,Latitude,Longitude,Confirmed,Recovered,Deaths
18101,France,2020-01-24,14.641500,-56.315900,2.0,0.0,0.0
18102,France,2020-01-25,14.641500,-56.315900,3.0,0.0,0.0
18103,France,2020-01-26,14.641500,-56.315900,3.0,0.0,0.0
18104,France,2020-01-27,14.641500,-56.315900,3.0,0.0,0.0
18105,France,2020-01-28,14.641500,-56.315900,4.0,0.0,0.0
...,...,...,...,...,...,...,...
18439,France,2020-12-27,91.463495,-238.646914,2616510.0,195861.0,62867.0
18440,France,2020-12-28,91.463495,-238.646914,2619616.0,196642.0,63235.0
18441,France,2020-12-29,91.463495,-238.646914,2631110.0,197726.0,64204.0
18442,France,2020-12-30,91.463495,-238.646914,2657624.0,198966.0,64508.0


This is how we can get the total number of Deaths from the daatset for France.

In [21]:
data_country_date.iloc[-1, 6] # 프랑스의 사망자 수를 제공합니다(5는 회복자 수, 4는 확진자 )

64759.0

[텍스트](https://docs.taipy.io/en/release-1.1/manuals/gui/viselements/text/) 시각적 요소를 사용합니다 . `{}` 사이에 어떤 Python 변수도 넣을 수 있지만 또한 어떠한 Python 코드도 넣을 수 있습니다.

![](img/control_text.png)

In [22]:
country_md = """
## Deaths <|{data_country_date.iloc[-1, 6]}|text|>

## Recovered <|{data_country_date.iloc[-1, 5]}|text|>

## Confirmed <|{data_country_date.iloc[-1, 4]}|text|>

<|{data_country_date}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|layout={layout}|options={options}|>
"""

In [23]:
gui.stop()
page.set_content(country_md)
gui.run(port=5005)

Gui server has been stopped
 * Server starting on http://127.0.0.1:5005
 * Serving Flask app 'Taipy'
 * Debug mode: off


## 로컬 on_change 
- 모든 _국가/지역_ 의 이름이 포함된 dropdown=True [selector](https://docs.taipy.io/en/latest/manuals/gui/viselements/selector/)를 추가합니다.
- _on_change_ selector 속성에 _on_change_country_ 함수의 이름을 지정 합니다 . 이 함수는 selector가 사용될 때 호출됩니다.
- 이 함수에는 'state' 매개변수가 있으며, 완성되어야 합니다. selector를 사용할 때 이 함수는 _state_ 인수와 함께 호출됩니다. 그것은 모든 Gui 변수를 포함합니다. 'state.data_country_date'는 GUI에서 사용되는 데이터 프레임입니다.

![](img/on_change_local.png)

In [24]:
selector_country = list(data['Country/Region'].astype(str).unique())
selected_country = 'France'

country_md = """
<|{selected_country}|selector|lov={selector_country}|on_change=on_change_country|dropdown|label=Country|>

## Deaths <|{data_country_date.iloc[-1, 6]}|>

## Recovered <|{data_country_date.iloc[-1, 5]}|>

## Confirmed <|{data_country_date.iloc[-1, 4]}|>

<|{data_country_date}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|layout={layout}|options={options}|>
"""

In [25]:
def on_change_country(state):
    # state는 모든 Gui 변수를 포함하며 이 state 변수를 통해 Gui를 업데이트할 수 있습니다.
    # state.selected_country, state.data_country_date, ...
    # 올바른 국가로 data_country_date 업데이트(initialize_case_evolution 사용)
    print("Chosen country: ", state.selected_country)
    state.data_country_date = initialize_case_evolution(data, state.selected_country)

In [26]:
gui.stop()
page.set_content(country_md)
gui.run(port=5006)

Gui server has been stopped
 * Server starting on http://127.0.0.1:5006
 * Serving Flask app 'Taipy'
 * Debug mode: off


## Layout

[layout](https://docs.taipy.io/en/latest/manuals/gui/viselements/layout/) 블록을 사용하여 페이지 구조를 변경합니다 . 이 블록은 텍스트/시각적 요소를 넣을 보이지 않는 열을 만듭니다.

구문:
```
<|layout|columns=1 1 1 ...|
(first column)

(in second column)

(third column)
(again, third column)

(...)
|>
```


In [27]:
final_country_md = """
<|layout|columns=1 1 1 1|
<|{selected_country}|selector|lov={selector_country}|on_change=on_change_country|dropdown|label=Country|>

## Deaths <|{data_country_date.iloc[-1, 6]}|>

## Recovered <|{data_country_date.iloc[-1, 5]}|>

## Confirmed <|{data_country_date.iloc[-1, 4]}|>
|>

<|{data_country_date}|chart|type=bar|x=Date|y[1]=Deaths|y[2]=Recovered|y[3]=Confirmed|layout={layout}|options={options}|>
"""

In [28]:
gui.stop()
page.set_content(final_country_md)
gui.run(port=5007)

Gui server has been stopped
 * Server starting on http://127.0.0.1:5007
 * Serving Flask app 'Taipy'
 * Debug mode: off


![](img/layout.png)

# Map

In [29]:
def initialize_map(data):
    data['Province/State'] = data['Province/State'].fillna(data["Country/Region"])
    data_province = data.groupby(["Country/Region",
                                  'Province/State',
                                  'Longitude',
                                  'Latitude'])\
                         .max()

    data_province_displayed = data_province[data_province['Deaths']>10].reset_index()

    data_province_displayed['Size'] = np.sqrt(data_province_displayed.loc[:,'Deaths']/data_province_displayed.loc[:,'Deaths'].max())*80 + 3
    data_province_displayed['Text'] = data_province_displayed.loc[:,'Deaths'].astype(str) + ' deaths </br> ' + data_province_displayed.loc[:,'Province/State']
    return data_province_displayed

In [30]:
data_province_displayed = initialize_map(data)
data_province_displayed.head()

,Country/Region,Province/State,Longitude,Latitude,Confirmed,Recovered,Deaths,Date,Size,Text
0,Afghanistan,Afghanistan,67.709953,33.93911,51526.0,41727.0,2191.0,2020-12-31,17.771247,2191.0 deaths </br> Afghanistan
1,Albania,Albania,20.168300,41.15330,58316.0,33634.0,1181.0,2020-12-31,13.844784,1181.0 deaths </br> Albania
2,Algeria,Algeria,1.659600,28.03390,99610.0,67127.0,2756.0,2020-12-31,19.566684,2756.0 deaths </br> Algeria
3,Andorra,Andorra,1.521800,42.50630,8049.0,7432.0,84.0,2020-12-31,5.892249,84.0 deaths </br> Andorra
4,Angola,Angola,17.873900,-11.20270,17553.0,11044.0,405.0,2020-12-31,9.350728,405.0 deaths </br> Angola


지도(map) 스타일을 지정하는 속성
- 마커(maker) color는 사망자 수 에 해당합니다(_Deaths_ 열 ).
- 마커(maker) size는 사망자 수에 따른 _Size_ 열의 크기에 해당합니다.

layout_map 매개변수로 지도의 초기 확대/축소(zoom) 및 위치(position)를 정의했습니다.

In [31]:
marker_map = {"color":"Deaths", "size": "Size", "showscale":True, "colorscale":"Viridis"}
layout_map = {
            "dragmode": "zoom",
            "mapbox": { "style": "open-street-map", "center": { "lat": 38, "lon": -90 }, "zoom": 3}
            }

우리는 Plotly에게 다음을 제공합니다:

- map type(유형)
- latitude(위도) 열의 이름
- longitude(경도) 열의 이름
- properties: 마커의 크기와 색상
- points(점)의 text에 대한 열 이름

In [32]:
map_md = """
<|{data_province_displayed}|chart|type=scattermapbox|selected={selected_points}|lat=Latitude|lon=Longitude|marker={marker_map}|layout={layout_map}|text=Text|mode=markers|height=800px|options={options}|>
"""

In [33]:
gui.stop()
page.set_content(map_md)
gui.run(port=5008)

Gui server has been stopped
 * Server starting on http://127.0.0.1:5008
 * Serving Flask app 'Taipy'
 * Debug mode: off


![](img/carte.png)

### 포인트 선택 시 API 호출 

지도에서 포인트를 선택 하면 API 호출을 통해 _expandable(확장 가능한)_ 블록에 데이터가 구현됩니다.

이를 위해서는 차트 의 _selected_ 속성이 필요하고 전역 _on_change_ 함수 를 통해 변경 사항을 처리해야 합니다.

In [34]:
map_md = """
<|Information|expandable|expanded=False|
<|layout|columns=1 1|
Country: <|{api_country}|>

Population: (Year: <|{api_year}|>, Value: <|{api_pop}|>)
|>
|>

<br/>
<|{data_province_displayed}|chart|type=scattermapbox|selected={selected_points}|lat=Latitude|lon=Longitude|marker={marker_map}|layout={layout_map}|text=Text|mode=markers|height=800px|options={options}|>
"""

In [35]:
url_pop = "https://countriesnow.space/api/v0.1/countries/population"

def api_call(state):
    state.api_country = state.data_province_displayed.loc[state.selected_points[0], "Country/Region"]
    
    if state.api_country == 'US':
        state.api_country = "United States"
    
    body = {"country": state.api_country}
    
    response_pop = requests.post(url_pop, body).json()
    
    if not response_pop['error']:
        state.api_year = response_pop['data']['populationCounts'][-1]['year']
        state.api_pop = response_pop['data']['populationCounts'][-1]['value']

In [36]:
def on_change(state, var_name, var_value):
    print(var_name, var_value)
     # on_change는 Gui 변수가 변경될 때마다 호출됩니다(Gui를 통하거나 state.xxx = yyy를 사용하여)
     # 1) selected_points 변경 시 'api_call' 호출
    if var_name == 'selected_points':
        api_call(state)

In [37]:
gui.stop()
gui.on_change = on_change
page.set_content(map_md)
gui.run(port=5009)

Gui server has been stopped
 * Server starting on http://127.0.0.1:5009
 * Serving Flask app 'Taipy'
 * Debug mode: off


# Part와 _render_ 속성 
- 'Map'과 'Country'의 lov(list of value:변수목록) [토글(toggle)](https://docs.taipy.io/en/latest/manuals/gui/viselements/toggle/)(selctor와 동일하게 작동)을 만듭니다.
- 토글 값에 따라 렌더링 여부를 결정하는 두 개의 part 블록을 만듭니다.
    - 이렇게 하려면 part 블록의 render 속성 `{}` 안에 Python 코드를 삽입할 수 있다는 사실을 이용하십시오.

In [38]:
representation_selector = ['Map', 'Country']
selected_representation = representation_selector[0]

In [39]:
main_page = """
<|{selected_representation}|toggle|lov={representation_selector}|>

<|part|render={selected_representation == "Country"}|
"""+final_country_md+"""
|>

<|part|render={selected_representation == "Map"}|
"""+map_md+"""
|>
""" 

In [40]:
gui.stop()
page.set_content(main_page)
gui.run(port=5010)

Gui server has been stopped
 * Server starting on http://127.0.0.1:5010
 * Serving Flask app 'Taipy'
 * Debug mode: off


![](img/part_render.png)

# Taipy Core
다음은 국가의 사망자 수를 예측하는 데 사용할 함수입니다. 우리는 다음을 진행할 것입니다:

- 데이터 전처리(_preprocess_),
- 훈련 및 테스트 데이터베이스 생성(_make_train_test_data_),
- 모델 훈련(_train_model_),
- 예측 생성(_forecast_),
- 과거 데이터와 예측(_result_) 으로 데이터 프레임을 생성합니다.


![](img/all_architecture.svg)

In [41]:
def preprocess(initial_data, country):
    data = initial_data.groupby(["Country/Region",'Date'])\
                       .sum()\
                       .dropna()\
                       .reset_index()

    preprocess_data = data.loc[data['Country/Region']==country].reset_index(drop=True)
    return preprocess_data
    
    
def make_train_test_data(preprocess_data):
    perc_test = 0.3
    nb_to_predict = int(perc_test*len(preprocess_data))
    train_data = preprocess_data[:-nb_to_predict]
    return train_data, nb_to_predict


def train_model(train_data):
    model = auto_arima(train_data['Deaths'],
                       start_p=1, start_q=1,
                       max_p=5, max_q=5,
                       start_P=0, seasonal=False,
                       d=1, D=1, trace=True,
                       error_action='ignore',  
                       suppress_warnings=True)
    model.fit(train_data['Deaths'])
    return model


def forecast(model, nb_to_predict):
    predictions = model.predict(n_periods=nb_to_predict)
    return predictions


def result(preprocess_data, predictions, nb_to_predict):
    pred_series = pd.Series([np.NaN]*(len(preprocess_data)-nb_to_predict) + list(predictions), name='Predictions')
    return pd.concat([preprocess_data, pred_series], axis=1)

먼저 Data Nodes를 정의한 다음 tasks(Python 함수와 연결됨)을 정의해야 합니다. 또한 이러한 tasks을 다른 pipelines으로 수집하고 이러한 pipelines을 scenario로 수집합니다.

Data Node에는 **unique id(고유 ID)** 가 필요합니다. 필요한 경우 CSV 및 SQL 용으로 스토리지 유형을 변경할 수 있습니다. 그런 다음 다른 매개변수(parameters)가 필요합니다.

### 전처리를 위한 데이터 노드 및 태스크(Data Nodes and Task for preprocess)

<img src="img/preprocess.svg" alt="drawing" width="500"/>

In [42]:
initial_data_cfg = Config.configure_data_node(id="initial_data",
                                              storage_type="csv",
                                              path=path_to_data,
                                              cacheable=True,
                                              validity_period=dt.timedelta(days=5))

country_cfg = Config.configure_data_node(id="country", default_data="France",
                                         cacheable=True, validity_period=dt.timedelta(days=5))

preprocess_data_cfg =  Config.configure_data_node(id="preprocess_data",
                                                  cacheable=True, validity_period=dt.timedelta(days=5))


task_preprocess_cfg = Config.configure_task(id="task_preprocess_data",
                                           function=preprocess,
                                           input=[initial_data_cfg, country_cfg],
                                           output=preprocess_data_cfg)

### make_train_test_data에 대한 Data Nodes 및 Task

<img src="img/make_train_test_data.svg" alt="drawing" width="500"/>

In [43]:
train_data_cfg =  Config.configure_data_node(id="train_data", cacheable=True, validity_period=dt.timedelta(days=5))

nb_to_predict_cfg = Config.configure_data_node(id="nb_to_predict",
                                               cacheable=True, validity_period=dt.timedelta(days=5))

task_train_test_cfg = Config.configure_task(id="task_make_train_test_data",
                                           function=make_train_test_data,
                                           input=preprocess_data_cfg,
                                           output=[train_data_cfg, nb_to_predict_cfg])

### train_model에 대한 Data Nodes 및 Task 

<img src="img/train_model.svg" alt="drawing" width="500"/>

In [44]:
model_cfg = Config.configure_data_node(id="model", cacheable=True, validity_period=dt.timedelta(days=5))

task_train_cfg = Config.configure_task(id="task_train",
                                      function=train_model,
                                      input=train_data_cfg,
                                      output=model_cfg)

### forecast에 대한 Data Nodes 및 Task

<img src="img/forecast.svg" alt="drawing" width="500"/>

In [45]:
predictions_cfg = Config.configure_data_node(id="predictions")

task_forecast_cfg = Config.configure_task(id="task_forecast",
                                      function=forecast,
                                      input=[model_cfg, nb_to_predict_cfg],
                                      output=predictions_cfg)

### result에 대한 Data Nodes 및 Task

<img src="img/result.svg" alt="drawing" width="500"/>

In [46]:
result_cfg = Config.configure_data_node(id="result")

task_result_cfg = Config.configure_task(id="task_result",
                                      function=result,
                                      input=[preprocess_data_cfg, predictions_cfg, nb_to_predict_cfg],
                                      output=result_cfg)

## [pipelines 구성(Configuration)](https://docs.taipy.io/en/release-1.1/manuals/reference/taipy.Config/#taipy.core.config.config.Config.configure_default_pipeline)

In [47]:
pipeline_preprocessing_cfg = Config.configure_pipeline(id="pipeline_preprocessing",
                                                       task_configs=[task_preprocess_cfg, task_train_test_cfg])

pipeline_train_cfg = Config.configure_pipeline(id="pipeline_train",
                                               task_configs=[task_train_cfg])

pipeline_forecast_cfg = Config.configure_pipeline(id="pipeline_forecast",
                                                       task_configs=[task_forecast_cfg, task_result_cfg])

## [scenario 구성(Configuration)](https://docs.taipy.io/en/release-1.1/manuals/reference/taipy.Config/#taipy.core.config.config.Config.configure_default_scenario)

In [48]:
scenario_cfg = Config.configure_scenario(id='scenario', pipeline_configs=[pipeline_preprocessing_cfg,
                                                                         pipeline_train_cfg,
                                                                         pipeline_forecast_cfg])

## scenario 생성(creation) 및 제출(submit)

In [49]:
scenario = tp.create_scenario(scenario_cfg, name='First Scenario')
tp.submit(scenario)

C:\ProgramData\Anaconda3\lib\site-packages\taipy\core\data\csv.py:145: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(self._path)


[2022-08-10 14:35:57,194][Taipy][INFO] job JOB_task_preprocess_data_9bfe56a3-53de-44e6-84b3-daffe8b01f15 is completed.
[2022-08-10 14:35:57,292][Taipy][INFO] job JOB_task_make_train_test_data_d9cc0e09-241f-4dc6-8b3c-cb6a024e7a93 is completed.
Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=2994.124, Time=0.25 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=3349.636, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=3045.305, Time=0.03 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=3200.566, Time=0.05 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=3402.421, Time=0.00 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=2996.090, Time=0.18 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=2996.081, Time=0.14 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=3162.541, Time=0.11 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=3014.299, Time=0.04 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=2996.306, Time=0.17 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=2992.816, Ti

In [50]:
scenario.model.read()

ARIMA(order=(1, 1, 1), scoring_args={}, suppress_warnings=True,
      with_intercept=False)

In [51]:
scenario.result.read()

,Country/Region,Date,Latitude,Longitude,Confirmed,Recovered,Deaths,Predictions
0,France,2020-01-24,14.641500,-56.315900,2.0,0.0,0.0,NaN
1,France,2020-01-25,14.641500,-56.315900,3.0,0.0,0.0,NaN
2,France,2020-01-26,14.641500,-56.315900,3.0,0.0,0.0,NaN
3,France,2020-01-27,14.641500,-56.315900,3.0,0.0,0.0,NaN
4,France,2020-01-28,14.641500,-56.315900,4.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...
338,France,2020-12-27,91.463495,-238.646914,2616510.0,195861.0,62867.0,32931.922629
339,France,2020-12-28,91.463495,-238.646914,2619616.0,196642.0,63235.0,32936.896584
340,France,2020-12-29,91.463495,-238.646914,2631110.0,197726.0,64204.0,32941.767217
341,France,2020-12-30,91.463495,-238.646914,2657624.0,198966.0,64508.0,32946.536673


In [52]:
scenario.predictions.read()

array([31336.9148603 , 31375.02135576, 31412.33627827, 31448.87607089,
       31484.65683512, 31519.69433801, 31554.00401905, 31587.60099703,
       31620.5000767 , 31652.71575524, 31684.26222872, 31715.15339829,
       31745.40287636, 31775.02399255, 31804.02979962, 31832.43307916,
       31860.24634727, 31887.48186005, 31914.15161901, 31940.26737635,
       31965.84064017, 31990.88267948, 32015.40452924, 32039.41699515,
       32062.93065847, 32085.95588065, 32108.50280792, 32130.58137572,
       32152.20131312, 32173.37214709, 32194.1032067 , 32214.40362723,
       32234.2823542 , 32253.74814729, 32272.80958425, 32291.47506462,
       32309.75281346, 32327.650885  , 32345.17716613, 32362.33937993,
       32379.14508903, 32395.601699  , 32411.71646154, 32427.49647773,
       32442.94870114, 32458.0799409 , 32472.89686468, 32487.40600167,
       32501.61374541, 32515.52635664, 32529.14996604, 32542.49057695,
       32555.55406799, 32568.34619569, 32580.87259697, 32593.13879169,
      

## 캐싱(Caching)
"input" Data Nodes 변경되지 않았기 때문에 일부 작업(job)을 건너뜁니다.

In [53]:
tp.submit(scenario)

[2022-08-10 14:35:59,198][Taipy][INFO] job JOB_task_preprocess_data_08204303-4ee6-46d1-9cce-987d682fa3d3 is skipped.
[2022-08-10 14:35:59,290][Taipy][INFO] job JOB_task_make_train_test_data_98a2592c-5109-4bd1-945e-30157cafbf86 is skipped.
[2022-08-10 14:35:59,363][Taipy][INFO] job JOB_task_train_287fab6e-e40e-47d1-a75d-2d2160eba695 is skipped.
[2022-08-10 14:35:59,476][Taipy][INFO] job JOB_task_forecast_fb520e40-8f86-433c-b1d4-b501f2e1fab7 is completed.
[2022-08-10 14:35:59,586][Taipy][INFO] job JOB_task_result_519372d8-37a7-4fcd-a1d3-ec33786ddc56 is completed.


## data nodes에 쓰기

데이터 노드를 쓰려면:

`<Data Node>.write(new_value)`

In [54]:
scenario.country.write('US')
tp.submit(scenario)
scenario.result.read()

C:\ProgramData\Anaconda3\lib\site-packages\taipy\core\data\csv.py:145: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(self._path)


[2022-08-10 14:36:00,865][Taipy][INFO] job JOB_task_preprocess_data_a420b732-3985-46eb-a823-0751d2035c80 is completed.
[2022-08-10 14:36:00,972][Taipy][INFO] job JOB_task_make_train_test_data_89ce8a20-8537-4558-bbcd-866b916bf410 is completed.
Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=3565.937, Time=0.03 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=3869.494, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=3564.625, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=3712.670, Time=0.06 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=4064.717, Time=0.00 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=3565.875, Time=0.03 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=3567.852, Time=0.05 sec
 ARIMA(1,1,0)(0,0,0)[0]             : AIC=3573.780, Time=0.01 sec

Best model:  ARIMA(1,1,0)(0,0,0)[0] intercept
Total fit time: 0.219 seconds
[2022-08-10 14:36:01,306][Taipy][INFO] job JOB_task_train_25b7a757-12e1-42da-9cad-9dda1822dfdb is completed.


,Country/Region,Date,Latitude,Longitude,Confirmed,Recovered,Deaths,Predictions
0,US,2020-01-22,47.140037,-120.791360,1.0,0.0,0.0,NaN
1,US,2020-01-23,47.140037,-120.791360,1.0,0.0,0.0,NaN
2,US,2020-01-24,85.345498,-210.450080,2.0,0.0,0.0,NaN
3,US,2020-01-25,87.000686,-209.922210,2.0,0.0,0.0,NaN
4,US,2020-01-26,158.375485,-442.264230,5.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...
340,US,2020-12-27,124119.377356,-298994.802493,19222064.0,0.0,334533.0,281684.137063
341,US,2020-12-28,124119.377356,-298994.802493,19396237.0,0.0,336438.0,282520.641619
342,US,2020-12-29,124119.377356,-298994.802493,19595117.0,0.0,340061.0,283357.146177
343,US,2020-12-30,124119.377356,-298994.802493,19827133.0,0.0,343783.0,284193.650735


## 간단한 framework

In [55]:
scenario = tp.create_scenario(scenario_cfg, name='Second Scenario')
tp.submit(scenario)

C:\ProgramData\Anaconda3\lib\site-packages\taipy\core\data\csv.py:145: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(self._path)


[2022-08-10 14:36:02,904][Taipy][INFO] job JOB_task_preprocess_data_2b7cfb73-2ee0-42e2-a464-2f9c06dd597b is completed.
[2022-08-10 14:36:03,014][Taipy][INFO] job JOB_task_make_train_test_data_b72bfdb4-8ea0-4710-b859-16cfb4374f40 is completed.
Performing stepwise search to minimize aic
 ARIMA(1,1,1)(0,0,0)[0] intercept   : AIC=2994.124, Time=0.11 sec
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=3349.636, Time=0.01 sec
 ARIMA(1,1,0)(0,0,0)[0] intercept   : AIC=3045.305, Time=0.04 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=3200.566, Time=0.05 sec
 ARIMA(0,1,0)(0,0,0)[0]             : AIC=3402.421, Time=0.00 sec
 ARIMA(2,1,1)(0,0,0)[0] intercept   : AIC=2996.090, Time=0.17 sec
 ARIMA(1,1,2)(0,0,0)[0] intercept   : AIC=2996.081, Time=0.14 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=3162.541, Time=0.11 sec
 ARIMA(2,1,0)(0,0,0)[0] intercept   : AIC=3014.299, Time=0.04 sec
 ARIMA(2,1,2)(0,0,0)[0] intercept   : AIC=2996.306, Time=0.15 sec
 ARIMA(1,1,1)(0,0,0)[0]             : AIC=2992.816, Ti

In [56]:
[s.model.read() for s in tp.get_scenarios()]

[ARIMA(order=(1, 1, 1), scoring_args={}, suppress_warnings=True,
       with_intercept=False),
 ARIMA(order=(1, 1, 0), scoring_args={}, suppress_warnings=True),
 ARIMA(order=(1, 1, 1), scoring_args={}, suppress_warnings=True,
       with_intercept=False)]

## 백엔드용 Gui 생성 
_scenario_selector_ 를 사용하면 시나리오를 선택하고 result를 표시할 수 있습니다.

In [57]:
scenario_selector = [(s.id, s.name) for s in tp.get_scenarios()]
selected_scenario = scenario.id
print(scenario_selector,'\n', selected_scenario)

[('SCENARIO_scenario_4ff955a8-31d8-4cde-9100-6c7a778462db', 'First Scenario'), ('SCENARIO_scenario_839d877f-6441-4b0c-8c94-7b208e2b21b4', 'First Scenario'), ('SCENARIO_scenario_e71a3874-8faa-401b-9799-80ec29f13e2e', 'Second Scenario')] 
 SCENARIO_scenario_e71a3874-8faa-401b-9799-80ec29f13e2e


In [58]:
result = scenario.result.read()
result

,Country/Region,Date,Latitude,Longitude,Confirmed,Recovered,Deaths,Predictions
0,France,2020-01-24,14.641500,-56.315900,2.0,0.0,0.0,NaN
1,France,2020-01-25,14.641500,-56.315900,3.0,0.0,0.0,NaN
2,France,2020-01-26,14.641500,-56.315900,3.0,0.0,0.0,NaN
3,France,2020-01-27,14.641500,-56.315900,3.0,0.0,0.0,NaN
4,France,2020-01-28,14.641500,-56.315900,4.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...
338,France,2020-12-27,91.463495,-238.646914,2616510.0,195861.0,62867.0,32931.922629
339,France,2020-12-28,91.463495,-238.646914,2619616.0,196642.0,63235.0,32936.896584
340,France,2020-12-29,91.463495,-238.646914,2631110.0,197726.0,64204.0,32941.767217
341,France,2020-12-30,91.463495,-238.646914,2657624.0,198966.0,64508.0,32946.536673


**팁** : value_by_id 속성이 선택 항목에 대해 True로 설정되면 selected_scenario가 튜플(tupple)의 첫 번째 요소(여기서는 id)를 직접 참조하게 됩니다.

![](img/predictions.png)

In [59]:
prediction_md = """
<|layout|columns=1 5 1 3|
<|{scenario_name}|input|label=Name|>

<br/>
<|Create|button|on_action=create_new_scenario|>

<|{selected_country}|selector|lov={selector_country}|dropdown|on_change=on_change_country|label=Country|>

<br/>
<|Submit|button|on_action=submit_scenario|>

<|{selected_scenario}|selector|lov={scenario_selector}|on_change=actualize_graph|dropdown|value_by_id|label=Scenario|>
|>

<|{result}|chart|x=Date|y[1]=Deaths|type[1]=bar|y[2]=Predictions|>
"""

In [60]:
def create_new_scenario(state):
    scenario = tp.create_scenario(scenario_cfg, name=state.scenario_name)
    state.scenario_selector += [(scenario.id, scenario.name)]

In [61]:
def actualize_graph(state):
    # 1) update the result dataframe
    # 2) change selected_country with the predicted country of the scenario
    scenario = tp.get(state.selected_scenario)
    state.result = scenario.result.read()
    state.selected_country = scenario.country.read()

In [62]:
def submit_scenario(state):
    # 1) get the selected scenario
    # 2) write in country Data Node, the selected country
    # 3) submit the scenario
    # 4) actualize le graph avec actualize_graph
    scenario = tp.get(state.selected_scenario)
    scenario.country.write(state.selected_country)
    tp.submit(scenario)
    actualize_graph(state)

In [63]:
gui.stop()
page.set_content(prediction_md)
gui.run(port=5011)

Gui server has been stopped
 * Server starting on http://127.0.0.1:5011


# 다중 페이지(Multi-pages) 및 Taipy Rest

다중 페이지 앱을 만들려면 names이 keys로, Markdowns이 values으로 포함된 사전(dictionary)만 있으면 됩니다.

_navbar_ 컨트롤 (<|navbar|>)에는 기본 동작(default behaviour)이 있습니다. 앱의 다른 페이지로 자동으로 리디렉션됩니다. 다른 솔루션도 있습니다.

![](img/multi_pages.png)

In [65]:
navbar_md = "<center>\n<|navbar|>\n</center>"

pages = {
    "Map":navbar_md+map_md,
    "Country":navbar_md+final_country_md,
    "Predictions":navbar_md+prediction_md
}

rest = tp.Rest()

gui_multi_pages = Gui(pages=pages)
tp.run(gui_multi_pages, rest, dark_mode=False, port=5012)

 * Server starting on http://127.0.0.1:5012
 * Serving Flask app 'taipy.rest.app'
 * Debug mode: off
